<img src="../imgs/logo.png" width="20%" align="right" style="margin:0px 20px">


# Evolutionary Computation

## 2.1 Genetic Algorithms

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://d9w.github.io/evolution/">https://d9w.github.io/evolution/</a>

## Outline

1. [Genetic Algorithm Overview](#overview)
2. [Initialization](#populations)
3. [Selection](#selection)
4. [Mutation](#mutation)
5. [Crossover](#crossover)
6. [Putting it all together](#together)
7. [Elitism](#elitism)

In [ ]:
using Random
using Statistics
using StatsBase
using Plots

## <a id="overview"></a>Genetic Algorithm Overview

Recall the outline of evolutionary algorithms that we discussed in section 1.2. The outline of an evolutionary algorithm is to start with an initial population and then to iterate through a cycle of evaluation, selection, and modification, passing the modified population on as the next generation.

<img src="../imgs/AlgoG.png">

This is the underlying algorithm of a genetic algorithm. This term is general and can cover a variety of more specific algorithms such as NSGA-II and NEAT, but in this section we'll discuss a classic genetic algorithm.

The defining features of a classic genetic algorithm are a large population initialized at random and the mutation and crossover operators during the modification step. In a classic GA, we will use a selection method to randomly select two individuals from the population, combine them using crossover, mutate the resulting individual, and add it to the new population. We will repeat this until the new population is full. Let's write that basic outline as a function.

In [ ]:
abstract type Individual end

function step_ga(population::Array{<:Individual})
    evaluate!(population)
    max_fit = maximum([i.fitness for i in population])
    
    new_population = Array{Individual}(undef, 0)
    while length(new_population) < length(population)
        parent1 = select(population)
        parent2 = select(population)
        child1 = crossover(parent1, parent2)
        child1 = mutate(child1)
        push!(new_population, child1)
    end
    
    new_population, max_fit
end

We've defined a single step of the genetic algorithm and can fill in the functions later. Let's now define the full evolution, keeping track of the maximum fitness in the population at each step.

In [ ]:
function ga(n_generations::Int)
    population = initialize()
    fits = zeros(n_generations)
    
    for i in 1:n_generations
        population, max_fit = step_ga(population)
        fits[i] = max_fit
    end
    
    fits
end

## <a id="initialization"></a>Initialization

Earlier we saw a new bit of Julia:
```
abstract type Individual end
```
This let us use the type `Array{Individual}` in our function definition even though a concrete definition of `Individual` hasn't been used yet. In the last notebook, we had to redefine functions in order to switch between the boolean `Individual` and the `FloatIndividual`.  Abstract types let us define functions which can work for either type, as long as they are subtypes of this abstract. This is known as multiple dispatch and we'll use it to define the later parts of our genetic algorithm. Let's define our two `Individual` types for now.

In [ ]:
mutable struct BitInd <: Individual
    genes::BitArray
    fitness::Float64
end

function BitInd(n::Int)
    BitInd(bitrand(n), -Inf)
end

In [ ]:
mutable struct FloatInd <: Individual
    genes::Array{Float64}
    fitness::Float64
end

function FloatInd(n::Int)
    FloatInd(rand(n), -Inf)
end

This will be very useful later, but for now just remember that we still have our two different types, `BitInd` and `FloatInd`, but that we can refer to both using `Individual`. Let's create population initialization methods for these two types.

In [ ]:
function bit_init(population_size::Int, n::Int)
    [BitInd(n) for i in 1:population_size]
end

function float_init(population_size::Int, n::Int)
    [FloatInd(n) for i in 1:population_size]
end

Let's see our different populations:

In [ ]:
population = bit_init(100, 4)
population[1:3]

In [ ]:
population = float_init(100, 4)
population[1:3]

## <a id="populations"></a>Evaluation

The first part of the genetic algorithm step is the evaluation. We'll look at two evaluation functions, one for our binary individual type, and one for the floating point individuals.

Looking at our genetic algorithm implementation, we use an `evaluate!` function to evaluate the entire population. Let's write that general function.

In [ ]:
function evaluate!(population::Array{<:Individual})
    for i in population
        i.fitness = objective(i)
    end
end

Note the `<:Individual` in our definition of `population`. This means that population is expected to be an `Array` of any subtype of `Individual`, meaning `BitInd` or `FloatInd`. This evaluate function will work for both types then. Let's write individual objective functions for these two different types. For the binary type, we'll use the OneMax function again.

In [ ]:
function objective(i::BitInd)
    sum(i.genes)
end

Let's test that.

In [ ]:
population = bit_init(10, 4)
evaluate!(population)
print([i.fitness for i in population])

For a floating point problem, let's look at something a bit more difficult than the sphere problem. Specifically, we'll use the [Rastrigin function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [ ]:
rastrigin(x) = 10*length(x) .+ sum(x.^2 .- 10 .* cos.(2*pi*x))

x = -5:0.1:5
y = -5:0.1:5
fz(x, y) = rastrigin([x, y])
plot(plot(x, y, fz, st=:surface), plot(x, y, fz, st=:contour), size = (800, 300))

We'll use a smaller section of the rastrigin function for demonstration, but you can also try the Genetic Algorithm on the entire space as an exercise.

In [ ]:
x = 0:0.01:1
y = 0:0.01:1
fz(x, y) = -rastrigin([x*5-2, y*5-2])
#fz(x, y) = exp(-rastrigin([x*5-2, y*5-2]))
plot(plot(x, y, fz, st=:surface), plot(x, y, fz, st=:contour), size = (800, 300))

Let's define our second objective function using this. Note that we'll also just call it `objective`, like the one we defined before. However, we'll be making a second function which applies to the `FloatInd` type.

In [ ]:
function objective(i::FloatInd)
    -rastrigin(i.genes .* 5 .- 2)
end

We can see the different functions we've defined used `methods`

In [ ]:
methods(objective)

Let's see what our first population of floating point individuals looks like in the Rastrigin function **search space**.

In [ ]:
population = float_init(100, 10)
xs = [i.genes[1] for i in population]
ys = [i.genes[2] for i in population]
plot(x, y, fz, st=:contour)
scatter!(xs, ys, label="pop", legend=:none)

As we can see, having a large population of individuals means we have a chance of already starting out in a good position. However, some others might be in **local maxima** - points which are higher than the areas around them, but not the **global maximum**. These points can be deceptive if we tend to select only the individuals in high points. What we want to do with selection in a genetic algorithm is maintain **diversity** while moving towards the **global maximum**. Let's see how we can do that.

<div class="alert alert-success">
    <b>Exercise 1</b>
    <br/>
Visualize the sphere function from the first class. Why do you think the Rastrigin function is more difficult than the sphere function? Is there any other function which looks more difficult?
    <br/>
</div>

## <a id="selection"></a>Selection

Which individuals should pass on their genetic information to the next generation? We could imagine a simple schemes of taking the best individuals globally, say 20% of them, and mutating each of these experts for the next generation. In a simple problem like this one, such a method might work. However, we would lose important genetic diversity, one of the main advantages of our large population. Let's explore some different selection methods. To do so, we'll need to evaluate all individuals.

For this first proposed method of taking the top individuals, which is called **truncation selection**, we'll need to order them. In Julia, we can override any function, including the basic functions of the language. To order our population, we can use the main `sort!` method, which relies on being able to compare individuals using `<`. So let's make a new definition for the `<`, or `isless`, function.

In [ ]:
import Base.isless

function isless(i1::Individual, i2::Individual)
    i1.fitness < i2.fitness
end
evaluate!(population)
println(population[1])
println(population[2])
println(population[1] < population[2])

Now that we have a way of comparing individuals, we can order lists of them using the `sort!` function. We'll sort from biggest to smallest to have the best individuals at the beginning of the population.

In [ ]:
population = float_init(100, 2)
evaluate!(population)
fits = [i.fitness for i in population]
println("First: ", population[1].fitness, ", last :", population[end].fitness)
println("Max: ", maximum(fits), ", min: ", minimum(fits))

In [ ]:
sort!(population, rev=true)
println("First: ", population[1].fitness, ", last :", population[end].fitness)
println("Max: ", maximum(fits), ", min: ", minimum(fits))

Let's plot where our best individuals are

In [ ]:
xs = [i.genes[1] for i in population]; ys = [i.genes[2] for i in population]
plot(x, y, fz, st=:contour)
scatter!(xs[21:end], ys[21:end], label="pop", legend=:outerright)
scatter!(xs[1:20], ys[1:20], label="experts")

As we can see, using this selection method, we risk optimizing towards local maxima, especially if we completely discard the rest of the population. Let's look at other selection methods for now, and we'll keep truncation selection in mind for later.

We will instead assign a probability to each individual based on their fitness. This is known as **fitness proportionate selection**. Specifically, we will use the probability

$p_i = \frac{f_i}{\sum_{j=1}^N f_j}$

The probability is each fitness divided by the sum. We'll use the `.` dot broadcast operator in Julia to divide each element of `fits` by the sum.

In [ ]:
fits = [i.fitness for i in population]
p = fits ./ sum(fits);

In [ ]:
histogram(p)

We can use these weights to sample from our population using the `sample` function from the `StatsBase` package

In [ ]:
sample(population, Weights(p))

Let's write this according to the `select` function definition we used before, which takes in the population and gives out a single individual.

In [ ]:
function fp_select(population::Array{<:Individual})
    fits = [-i.fitness for i in population]
    fits = maximum(fits) .- fits
    p = fits ./ sum(fits)
    sample(population, Weights(p))
end

Does this perform better than our previous method? Let's select 20 individuals using this method.

In [ ]:
selected = Array{Individual}(undef, 20)
for i in eachindex(selected)
    selected[i] = fp_select(population)
end

In [ ]:
sxs = [i.genes[1] for i in selected]
sys = [i.genes[2] for i in selected]
xs = [i.genes[1] for i in population]
ys = [i.genes[2] for i in population]
plot(x, y, fz, st=:contour)
scatter!(xs, ys, label="pop", legend=:outertopright)
scatter!(sxs, sys, label="fp_select")

As we can see here, individuals are not only found in local maxima but throughout the search space, which is good for diversity. The last selection method we'll use is called tournament selection. This method creates small random tournaments and selects the winner from this smaller subset for every new individual. We'll use a tournament size of 3, so we'll randomly select 3 individuals and then return the best individual from those 3.

In [ ]:
function tournament_select(population::Array{<:Individual})
    tournament = sample(population, 3)
    sort!(tournament, rev=true)[1]
end

Let's look at 20 selections from the tournament selection method and compare it to the fitness proportionate method.

In [ ]:
winners = Array{Individual}(undef, 20)
for i in eachindex(winners)
    winners[i] = tournament_select(population)
end

In [ ]:
wxs = [i.genes[1] for i in winners]
wys = [i.genes[2] for i in winners]
p1 = plot(x, y, fz, st=:contour)
scatter!(xs, ys, legend=:none)
scatter!(sxs, sys)
title!("Fitness Proportionate")
p2 = plot(x, y, fz, st=:contour)
scatter!(xs, ys, legend=:none)
scatter!(wxs, wys)
title!("Tournament")
plot(p1, p2, size=(950, 400))

<div class="alert alert-success">
    <b>Exercise 2</b>
    <br/>
    Plot the histogram of fitness values of 100 selected individuals using these two methods, and using a larger $n$ value. How do the distributions from the two methods compare? Try it also for a population of binary individuals on the OneMax function. Finally, try increasing the tournament size. What effect does that have on the selected individuals' fitness distribution?
    <br/>
</div>

## <a id="mutation"></a>Mutation

For this genetic algorithm, we'll simply reuse the functions we defined in section 1.3. We will flip the gene bit for binary individuals and use `rand` for floating point individuals.

In [ ]:
function mutate(ind::BitInd; mutation_rate::Float64=1.0/length(ind.genes))
    new_genes = copy(ind.genes)
    for i in eachindex(new_genes)
        if rand() < mutation_rate
            new_genes[i] = ~ind.genes[i]
        end
    end
    BitInd(new_genes, -Inf)
end

In [ ]:
function mutate(ind::FloatInd; mutation_rate::Float64=1.0/length(ind.genes))
    new_genes = copy(ind.genes)
    for i in eachindex(new_genes)
        if rand() < mutation_rate
            new_genes[i] = rand()
        end
    end
    FloatInd(new_genes, -Inf)
end

Let's test it

In [ ]:
println(population[1])
println(mutate(population[1]))

<div class="alert alert-success">
    <b>Exercise 3</b>
    <br/>
    Our mutation operator samples from the entire space. Can we instead use the previous gene value as a starting point for mutation? Implement a new mutation operator that adds a small random value between -0.1 and 0.1 to the modified gene.
    <br/>
</div>

## <a id="crossover"></a>Crossover

Considering we have such a large population, is there some way to combine individual solutions to lead to better solutions? For example, could we make an individual which inherits information from two parent individuals? This is the idea behind crossover, the other operator in genetic algorithms besides mutation. It is based on sexual reproduction where the genetic information of two parent individuals is mixed to create an offspring individual. The idea of combining the information from multiple individuals together to create the next generation is something we'll explore in more detail next class when discussing evolutionary strategies. For now, let's look at ways to combine two individuals.

<img src="../imgs/crossover.png" width="80%" height="auto">

The first methods we'll look at is single-point crossover. Here, the child genes are composed of two continuous sections, the first from one parent and the second from the other parent.

In [ ]:
function one_point_crossover(p1::Individual, p2::Individual)
    child = copy(p1.genes)
    n = rand(1:length(p2.genes))
    child[n:end] = copy(p2.genes[n:end])
    typeof(p1)(child, -Inf)
end

Let's test that

In [ ]:
p1 = BitInd(15)
p2 = BitInd(15)
println("P1: ", p1)
println("P2: ", p2)
child = one_point_crossover(p1, p2)
println("C : ", child)

This is most useful when sequential sections of a genome should be passed together, which is the case in biological evolution but not always necessary in artificial evolution. This method is also the basis of k-point crossover, which follows the same method but splits at $k$ points, alternating between parents at each crossing point. Note that this method can generate two children at once, but for the sake of coherence we'll just use the first child.

The second crossover method we'll look at is uniform crossover, which randomly chooses a different parent for each gene.

In [ ]:
function uniform_crossover(p1::Individual, p2::Individual)
    child = copy(p1.genes)
    for i in eachindex(child)
        if rand() < 0.5
            child[i] = p2.genes[i]
        end
    end
    typeof(p1)(child, -Inf)
end

And the test

In [ ]:
p1 = BitInd(15)
p2 = BitInd(15)
println("P1: ", p1)
println("P2: ", p2)
child = one_point_crossover(p1, p2)
println("C : ", child)

<div class="alert alert-success">
    <b>Exercise 4</b>
    <br/>
    Which crossover method do you expect to work better for the OneMax problem? What about for the Rastrigin function? Explain your reasoning.
    <br/>
</div>

## <a id="together"></a>Putting it all together

Let's look back at the functions we defined earlier

In [ ]:
function step_ga(population::Array{<:Individual})
    evaluate!(population)
    max_fit = maximum([i.fitness for i in population])
    
    new_population = Array{Individual}(undef, 0)
    while length(new_population) < length(population)
        parent1 = select(population)
        parent2 = select(population)
        child1 = crossover(parent1, parent2)
        child1 = mutate(child1)
        push!(new_population, child1)
    end
    
    new_population, max_fit
end

In [ ]:
function ga(n_generations::Int)
    population = initialize()
    fits = zeros(n_generations)
    
    for i in 1:n_generations
        population, max_fit = step_ga(population)
        fits[i] = max_fit
    end
    
    fits
end

We need to define our function choices for `initialize`, `select`, and `crossover`. Let's focus on the Rastrigin function and use uniform crossover. We'll compare the two selection methods, starting with fitness proporitionate selection.

In [ ]:
initialize() = float_init(100, 10)
crossover = uniform_crossover

In [ ]:
select = fp_select
fits_fp = zeros(100, 10)
for i in 1:10
    fits_fp[:, i] = ga(100)
    println(i, " ", fits_fp[end, i])
end

In [ ]:
select = tournament_select
fits_tourney = zeros(100, 10)
for i in 1:10
    fits_tourney[:, i] = ga(100)
    println(i, " ", fits_tourney[end, i])
end

In [ ]:
μ_fp = mean(fits_fp, dims=2)
σ_fp = std(fits_fp, dims=2)
μ_tourney = mean(fits_tourney, dims=2)
σ_tourney = std(fits_tourney, dims=2)
plot(μ_fp, ribbon=σ_fp, label="FP", legend=:outertopright)
plot!(μ_tourney, ribbon=σ_tourney, label="Tournament")

While the randomness of tournament selection may appear disadvantageous, the diversity it provides in the search allows us to continue looking through new areas where we might get stuck with fitness proportionate selection. Tournament selection also gaurantees improvement by selecting individuals that are at least better than two other individuals in the population, while fitness proportionate selection may randomly choose bad individuals often.

Our results seem to fluctuate quite a bit. Why is that? We didn't see that before with the $(1+1)$ and $(1+\lambda)$ EAs...

## <a id="elitism"></a>Elitism

An advantage that the $(1+1)$ and $(1+\lambda)$ EAs have over the genetic algorithm we've coded here is that their convergence is **monotonically increasing**. This means as evolution goes on, the result will only improve and never get worse. This is because the best fitness is only ever removed when the expert is replaced by an equally good or better individual. However, neither of the selection methods we compared have this guarantee: the best individual can leave the population easily! While completely basing our selection on global competition may not be a good practice, we do want to keep at least the best individual. This is known as **elitism** and is the practice of directly passing on a certain number of individuals *without mutation* into the next generation. This is similar to the truncation selection we used at the beginning, but the individuals selected from truncation selection will bypass the modification step entirely.

In [ ]:
num_elites = 5
select = tournament_select

function step_ga(population::Array{<:Individual})
    evaluate!(population)
    sort!(population, rev=true)
    max_fit = maximum([i.fitness for i in population])
    
    new_population = Array{Individual}(undef, 0)
    append!(new_population, population[1:num_elites])
    while length(new_population) < length(population)
        parent1 = select(population)
        parent2 = select(population)
        child1 = crossover(parent1, parent2)
        child1 = mutate(child1)
        push!(new_population, child1)
    end
    
    new_population, max_fit
end

In [ ]:
select = fp_select
fits_fp = zeros(100, 10)
for i in 1:10
    fits_fp[:, i] = ga(100)
    println(i, " ", fits_fp[end, i])
end

In [ ]:
select = tournament_select
fits_tourney = zeros(100, 10)
for i in 1:10
    fits_tourney[:, i] = ga(100)
    println(i, " ", fits_tourney[end, i])
end

In [ ]:
μ_fp = mean(fits_fp, dims=2)
σ_fp = std(fits_fp, dims=2)
μ_tourney = mean(fits_tourney, dims=2)
σ_tourney = std(fits_tourney, dims=2)
plot(μ_fp, ribbon=σ_fp, label="FP", legend=:outertopright)
plot!(μ_tourney, ribbon=σ_tourney, label="Tournament")

<div class="alert alert-success">
    <b>Exercise 5</b>
    <br/>
We now have many parameter choices: population size, mutation function, mutation rate, tournament size if using tournament selection, crossover function, number of elites. Using a maximum of 10000 <b>evaluations</b> (ie 100 generations with a population of 100), compare these parameter choices. What is the best value you can reach with $n=10$? Do the parameter results you get generalize to higher values of $n$? The best value should include the average and standard deviation over multiple runs.
    <br/>
</div>

<div class="alert alert-success">
    <b>Exercise 6</b>
    <br/>
Run the same analysis for the binary problem OneMax. Compare this result to the $(1+1)$ and $(1+\lambda)$ EAs from the first class. Which method is the best?
    <br/>
</div>